# ANOVOS - Association Evaluator
Following notebook shows the list of functions related to "asociation evaultion" module provided under ANOVOS package and how it can be invoked accordingly.
- [Correlation Matrix](#Correlation-Matrix)
- [Variable Clustering](#Variable-Clustering)
- [Information Value (IV)](#Information-Value-(IV))
- [Information Gain (IG)](#Information-Gain-(IG))

**Setting Spark Session**

In [1]:
#set run type variable
run_type = "local" # "local", "emr", "databricks", "ak8s"

In [2]:
#For run_type Azure Kubernetes, run the following block 
import os
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

if run_type == "ak8s":
    fs_path="<insert conf spark.hadoop.fs master url here> ex: spark.hadoop.fs.azure.sas.<container>.<account_name>.blob.core.windows.net"
    auth_key="<insert value of sas_token here>"
    master_url="<insert kubernetes master url path here> ex: k8s://"
    docker_image="<insert name docker image here>"
    kubernetes_namespace ="<insert kubernetes namespace here>"

    # Create Spark config for our Kubernetes based cluster manager
    sparkConf = SparkConf()
    sparkConf.setMaster(master_url)
    sparkConf.setAppName("Anovos_pipeline")
    sparkConf.set("spark.submit.deployMode","client")
    sparkConf.set("spark.kubernetes.container.image", docker_image)
    sparkConf.set("spark.kubernetes.namespace", kubernetes_namespace)
    sparkConf.set("spark.executor.instances", "4")
    sparkConf.set("spark.executor.cores", "4")
    sparkConf.set("spark.executor.memory", "16g")
    sparkConf.set("spark.kubernetes.pyspark.pythonVersion", "3")
    sparkConf.set("spark.kubernetes.authenticate.driver.serviceAccountName", "spark")
    sparkConf.set(fs_path,auth_key)
    sparkConf.set("spark.kubernetes.authenticate.serviceAccountName", "spark")
    sparkConf.set("spark.jars.packages", "org.apache.hadoop:hadoop-azure:3.2.0,com.microsoft.azure:azure-storage:8.6.3,io.github.histogrammar:histogrammar_2.12:1.0.20,io.github.histogrammar:histogrammar-sparksql_2.12:1.0.20,org.apache.spark:spark-avro_2.12:3.2.1")

    # Initialize our Spark cluster, this will actually
    # generate the worker nodes.
    spark = SparkSession.builder.config(conf=sparkConf).getOrCreate()
    sc = spark.sparkContext

#For other run types import from anovos.shared.
else:
    from anovos.shared.spark import *
    auth_key = "NA"

2022-06-07 11:54:39.501 | INFO     | anovos.shared.spark:init_spark:54 - Getting spark session, context and sql context app_name: Anovos_pipeline


:: loading settings :: url = jar:file:/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/mobilewalla/.ivy2/cache
The jars for the packages stored in: /Users/mobilewalla/.ivy2/jars
io.github.histogrammar#histogrammar_2.12 added as a dependency
io.github.histogrammar#histogrammar-sparksql_2.12 added as a dependency
org.apache.spark#spark-avro_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-0185f22c-7c52-427c-8a54-daa46bb27e9b;1.0
	confs: [default]
	found io.github.histogrammar#histogrammar_2.12;1.0.20 in central
	found io.github.histogrammar#histogrammar-sparksql_2.12;1.0.20 in central
	found org.apache.spark#spark-avro_2.12;3.2.1 in central
	found org.tukaani#xz;1.8 in central
	found org.spark-project.spark#unused;1.0.0 in central
:: resolution report :: resolve 269ms :: artifacts dl 9ms
	:: modules in use:
	io.github.histogrammar#histogrammar-sparksql_2.12;1.0.20 from central in [default]
	io.github.histogrammar#histogrammar_2.12;1.0.20 from central in [default]
	org.apache.spark#spark-avro_2.12;

In [3]:
sc.setLogLevel("ERROR")
import warnings
warnings.filterwarnings('ignore')

**Input/Output Path**

In [4]:
inputPath = "../data/income_dataset/csv"
outputPath = "../output/income_dataset/data_analyzer"

In [5]:
from anovos.data_ingest.data_ingest import read_dataset

In [6]:
df = read_dataset(spark, file_path = inputPath, file_type = "csv",file_configs = {"header": "True", 
                                                                           "delimiter": "," , 
                                                                           "inferSchema": "True"})
df = df.drop("dt_1", "dt_2")
df.toPandas().head(5)

ifa   age         workclass    fnlwgt    logfnl empty  education  \
0  1a   NaN         State-gov   77516.0  4.889391  None  Bachelors   
1  2a   NaN  Self-emp-not-inc   83311.0  4.920702  None  Bachelors   
2  3a  38.0           Private  215646.0  5.333741  None    HS-grad   
3  4a  53.0           Private  234721.0  5.370552  None       11th   
4  5a   NaN           Private  338409.0  5.529442  None  Bachelors   

   education-num      marital-status         occupation   relationship   race  \
0           13.0       Never-married       Adm-clerical  Not-in-family  White   
1           13.0  Married-civ-spouse    Exec-managerial        Husband  White   
2            9.0            Divorced  Handlers-cleaners  Not-in-family  White   
3            7.0  Married-civ-spouse  Handlers-cleaners        Husband  Black   
4           13.0  Married-civ-spouse     Prof-specialty           Wife  Black   

      sex  capital-gain  capital-loss  hours-per-week native-country income  
0    Male        2174.0           0.0            40.0   UnitedStates  <=50K  
1    Male           0.0           0.0            13.0   UnitedStates  <=50K  
2    Male           0.0           0.0            40.0   UnitedStates  <=50K  
3    Male           0.0           0.0            40.0   UnitedStates  <=50K  
4  Female           0.0           0.0            40.0           Cuba  <=50K

# Correlation Matrix
- API specification of function **correlation_matrix** can be found <a href="https://docs.anovos.ai/api/data_analyzer/association_evaluator.html">here</a>

In [7]:
from anovos.data_analyzer.association_evaluator import correlation_matrix

In [8]:
# Example 1 - 'all' columns (excluding drop_cols) --- MUST remove high cardinality columns
odf = correlation_matrix(spark, idf = df, list_of_cols='all', drop_cols=['ifa'])
odf.toPandas()

2022-06-06 19:24:45,995 INFO [histogram_filler_base]: Filling 136 specified histograms. auto-binning.
100%|█████████████████████████████████████████████████████████████| 136/136 [00:08<00:00, 16.43it/s]


attribute       age  capital-gain  capital-loss  education  \
0              age  1.000000      0.210563      0.136960   0.351468   
1     capital-gain  0.210563      1.000000      0.000000   0.232282   
2     capital-loss  0.136960      0.000000      1.000000   0.137968   
3        education  0.351468      0.232282      0.137968   1.000000   
4    education-num  0.307710      0.278753      0.097767   0.999993   
5           fnlwgt  0.093912      0.000000      0.000000   0.062800   
6   hours-per-week  0.374326      0.134373      0.067687   0.214868   
7           income  0.371282      0.363159      0.236992   0.408941   
8           logfnl  0.095748      0.000000      0.000000   0.084964   
9   marital-status  0.567636      0.147185      0.139274   0.615107   
10  native-country  0.090161      0.000000      0.024224   0.416369   
11      occupation  0.317458      0.150385      0.075212   0.495816   
12            race  0.037570      0.000000      0.000000   0.250895   
13    relationship  0.518391      0.220468      0.133671   0.385173   
14             sex  0.136002      0.077803      0.090275   0.111322   
15       workclass  0.299177      0.104854      0.050194   0.234612   

    education-num    fnlwgt  hours-per-week    income    logfnl  \
0        0.307710  0.093912        0.374326  0.371282  0.095748   
1        0.278753  0.000000        0.134373  0.363159  0.000000   
2        0.097767  0.000000        0.067687  0.236992  0.000000   
3        0.999993  0.062800        0.214868  0.408941  0.084964   
4        1.000000  0.064602        0.217432  0.467219  0.087599   
5        0.064602  1.000000        0.030981  0.045367  0.960338   
6        0.217432  0.030981        1.000000  0.310192  0.079669   
7        0.467219  0.045367        0.310192  1.000000  0.067692   
8        0.087599  0.960338        0.079669  0.067692  1.000000   
9        0.193128  0.069952        0.264090  0.591336  0.059919   
10       0.421516  0.172020        0.073563  0.115731  0.196478   
11       0.590182  0.079118        0.346109  0.446535  0.110344   
12       0.119653  0.134013        0.114166  0.131476  0.187069   
13       0.227612  0.037772        0.322045  0.452741  0.064678   
14       0.112019  0.018828        0.323359  0.323706  0.031693   
15       0.227179  0.061078        0.279103  0.230552  0.080529   

    marital-status  native-country  occupation      race  relationship  \
0         0.567636        0.090161    0.317458  0.037570      0.518391   
1         0.147185        0.000000    0.150385  0.000000      0.220468   
2         0.139274        0.024224    0.075212  0.000000      0.133671   
3         0.615107        0.416369    0.495816  0.250895      0.385173   
4         0.193128        0.421516    0.590182  0.119653      0.227612   
5         0.069952        0.172020    0.079118  0.134013      0.037772   
6         0.264090        0.073563    0.346109  0.114166      0.322045   
7         0.591336        0.115731    0.446535  0.131476      0.452741   
8         0.059919        0.196478    0.110344  0.187069      0.064678   
9         1.000000        0.152067    0.351355  0.246052      0.713109   
10        0.152067        1.000000    0.229639  0.630329      0.166632   
11        0.351355        0.229639    1.000000  0.154366      0.528692   
12        0.246052        0.630329    0.154366  1.000000      0.600369   
13        0.713109        0.166632    0.528692  0.600369      1.000000   
14        0.547093        0.078009    0.716283  0.608577      0.890410   
15        0.201391        0.458213    0.741602  0.085917      0.181764   

         sex  workclass  
0   0.136002   0.299177  
1   0.077803   0.104854  
2   0.090275   0.050194  
3   0.111322   0.234612  
4   0.112019   0.227179  
5   0.018828   0.061078  
6   0.323359   0.279103  
7   0.323706   0.230552  
8   0.031693   0.080529  
9   0.547093   0.201391  
10  0.078009   0.458213  
11  0.716283   0.741602  
12  0.608577   0.085917  
13  0.890410   0.181764  
14  1.000000

In [9]:
# Example 2 - selected columns
odf = correlation_matrix(spark, idf = df, list_of_cols= ['age','sex','race','workclass','fnlwgt'])
odf.toPandas()

2022-06-06 19:24:57,129 INFO [histogram_filler_base]: Filling 15 specified histograms. auto-binning.
100%|███████████████████████████████████████████████████████████████| 15/15 [00:00<00:00, 21.68it/s]


attribute       age    fnlwgt      race       sex  workclass
0        age  1.000000  0.093912  0.037570  0.136002   0.299177
1     fnlwgt  0.093912  1.000000  0.134013  0.018828   0.061078
2       race  0.037570  0.134013  1.000000  0.608577   0.085917
3        sex  0.136002  0.018828  0.608577  1.000000   0.186017
4  workclass  0.299177  0.061078  0.085917  0.186017   1.000000

In [10]:
# Example 3 - selected columns + presaved stats
from anovos.data_analyzer.stats_generator import measures_of_cardinality
from anovos.data_ingest.data_ingest import write_dataset
unique = write_dataset(measures_of_cardinality(spark, df),outputPath+"/unique","parquet", file_configs={"mode":"overwrite"})

odf = correlation_matrix(spark, idf = df, list_of_cols= ['age','sex','race','workclass','fnlwgt'],
                                  stats_unique={"file_path":outputPath+"/unique", "file_type": "parquet"})
odf.toPandas()

2022-06-06 19:25:02,820 INFO [histogram_filler_base]: Filling 15 specified histograms. auto-binning.
100%|███████████████████████████████████████████████████████████████| 15/15 [00:00<00:00, 19.70it/s]


attribute       age    fnlwgt      race       sex  workclass
0        age  1.000000  0.093912  0.037570  0.136002   0.299177
1     fnlwgt  0.093912  1.000000  0.134013  0.018828   0.061078
2       race  0.037570  0.134013  1.000000  0.608577   0.085917
3        sex  0.136002  0.018828  0.608577  1.000000   0.186017
4  workclass  0.299177  0.061078  0.085917  0.186017   1.000000

# Variable Clustering
- API specification of function **variable_clustering** can be found <a href="https://docs.anovos.ai/api/data_analyzer/association_evaluator.html">here</a>
- Valid only on smaller dataset which can fit into pandas dataframe. Sample size can controlled by sample_size argument (default value: 100,000)

In [11]:
from anovos.data_analyzer.association_evaluator import variable_clustering

In [12]:
# Example 1 - with mandatory arguments (rest arguments have default values)
odf = variable_clustering(spark, df)
odf.toPandas()

Cluster       Attribute  RS_Ratio
0         0  hours-per-week    0.8264
1         0  marital-status    0.4999
2         0    relationship    0.3526
3         0             sex    0.3369
4         1          logfnl    0.2273
5         1          fnlwgt    0.2277
6         2          income    0.5764
7         2   education-num    0.4163
8         2      occupation    0.5893
9         2    capital-loss    0.8975
10        3            race    0.3778
11        3  native-country    0.3735
12        4             age    0.4740
13        4       education    0.8361
14        4       workclass    0.5254
15        5    capital-gain    0.0000
16        6             ifa    0.0000

In [13]:
# Example 2 - 'all' columns (excluding drop_cols)
odf = variable_clustering(spark, idf = df, list_of_cols='all', drop_cols=['ifa'])
odf.toPandas()

Cluster       Attribute  RS_Ratio
0         0  hours-per-week    0.8264
1         0  marital-status    0.4999
2         0    relationship    0.3526
3         0             sex    0.3369
4         1          logfnl    0.2271
5         1          fnlwgt    0.2277
6         2          income    0.5764
7         2   education-num    0.4163
8         2      occupation    0.5893
9         2    capital-loss    0.8975
10        3            race    0.3778
11        3  native-country    0.3735
12        4             age    0.4740
13        4       education    0.8361
14        4       workclass    0.5254
15        5    capital-gain    0.0000

In [14]:
# Example 3 - selected columns
odf = variable_clustering(spark, idf = df, list_of_cols= ['age','sex','race','workclass','fnlwgt'])
odf.toPandas()

Cluster  Attribute  RS_Ratio
0        0  workclass    0.5303
1        0        age    0.4587
2        0     fnlwgt    0.8270
3        1       race    0.4607
4        1        sex    0.4618

In [15]:
# Example 4 - only numerical columns (user warning is shown as encoding was not required due to absence of any categorical column)
odf = variable_clustering(spark, idf = df, list_of_cols= ['age','education-num','capital-gain'])
odf.toPandas()

Cluster      Attribute  RS_Ratio
0        0            age    0.7640
1        0   capital-gain    0.4886
2        0  education-num    0.5839

In [16]:
# Example 5 - only categorical columns
odf = variable_clustering(spark, idf = df, list_of_cols= ['sex','race','workclass'])
odf.toPandas()

Cluster  Attribute  RS_Ratio
0        0       race    0.4606
1        0        sex    0.4605
2        1  workclass    0.0000

In [17]:
# Example 6 - Change in Sample Size
odf = variable_clustering(spark, idf = df, list_of_cols= 'all', sample_size=10000)
odf.toPandas()

Cluster       Attribute  RS_Ratio
0         0  hours-per-week    0.8462
1         0  marital-status    0.4953
2         0    relationship    0.3586
3         0             sex    0.3328
4         1          logfnl    0.2217
5         1          fnlwgt    0.2224
6         2          income    0.5328
7         2   education-num    0.4581
8         2      occupation    0.6609
9         2    capital-loss    0.8752
10        3  native-country    0.3771
11        3            race    0.3798
12        4             age    0.4295
13        4       workclass    0.4218
14        5    capital-gain    0.0000
15        6       education    0.0000
16        7             ifa    0.0000

In [18]:
# Example 7 - selected columns + presaved stats
from anovos.data_analyzer.stats_generator import measures_of_cardinality, measures_of_centralTendency
from anovos.data_ingest.data_ingest import write_dataset
unique = write_dataset(measures_of_cardinality(spark, df),outputPath+"/unique","parquet", file_configs={"mode":"overwrite"})
mode = write_dataset(measures_of_centralTendency(spark, df),outputPath+"/mode","parquet", file_configs={"mode":"overwrite"})

odf = variable_clustering(spark, idf = df, list_of_cols= ['age','sex','race','workclass','fnlwgt'],
                                  stats_unique={"file_path":outputPath+"/unique", "file_type": "parquet"},
                                  stats_mode={"file_path":outputPath+"/mode", "file_type": "parquet"})
odf.toPandas()

Cluster  Attribute  RS_Ratio
0        0  workclass    0.5303
1        0        age    0.4587
2        0     fnlwgt    0.8270
3        1       race    0.4607
4        1        sex    0.4618

# Information Value (IV)
- API specification of function **IV_calculation** can be found <a href="https://docs.anovos.ai/api/data_analyzer/association_evaluator.html">here</a>
- Supports only binary target variable

In [19]:
from anovos.data_analyzer.association_evaluator import IV_calculation

In [20]:
# Example 1 - with mandatory arguments (rest arguments have default values)
odf = IV_calculation(spark, df, label_col='income', event_label=">50K")
odf.toPandas()

attribute      iv
0     relationship  1.5348
1   marital-status  1.3390
2              age  1.0704
3       occupation  0.7772
4        education  0.7345
5    education-num  0.6984
6   hours-per-week  0.4563
7     capital-gain  0.3138
8              sex  0.3037
9        workclass  0.1625
10    capital-loss  0.0860
11  native-country  0.0780
12            race  0.0697
13          fnlwgt  0.0087
14          logfnl  0.0048
15           empty  0.0000
16             ifa     NaN

In [21]:
# Example 2 - 'all' columns (excluding drop_cols)
odf = IV_calculation(spark, idf = df, list_of_cols='all', drop_cols=['ifa'], label_col='income', event_label=">50K")
odf.toPandas()

attribute      iv
0     relationship  1.5348
1   marital-status  1.3390
2              age  1.0704
3       occupation  0.7772
4        education  0.7345
5    education-num  0.6984
6   hours-per-week  0.4563
7     capital-gain  0.3138
8              sex  0.3037
9        workclass  0.1625
10    capital-loss  0.0860
11  native-country  0.0780
12            race  0.0697
13          fnlwgt  0.0087
14          logfnl  0.0048
15           empty  0.0000

In [22]:
# Example 3 - selected columns
odf = IV_calculation(spark, idf = df, list_of_cols= ['age','sex','race','workclass','fnlwgt'], label_col='income', event_label=">50K")
odf.toPandas()

attribute      iv
0        age  1.0704
1        sex  0.3037
2  workclass  0.1625
3       race  0.0697
4     fnlwgt  0.0087

In [23]:
# Example 4 - selected columns + encoding configs (bin method equal_range instead of default equal_frequency )
odf = IV_calculation(spark, idf = df, list_of_cols= ['age','sex','race','workclass','fnlwgt'], label_col='income', 
                    event_label=">50K", encoding_configs={'bin_method': 'equal_range', 
                                                          'bin_size': 10, 'monotonicity_check': 0})
odf.toPandas()

attribute      iv
0        age  1.0436
1        sex  0.3037
2  workclass  0.1625
3       race  0.0697
4     fnlwgt  0.0016

In [24]:
# Example 5 - selected columns + encoding configs (bin_size 20 instead of default 10 )
odf = IV_calculation(spark, idf = df, list_of_cols= ['age','sex','race','workclass','fnlwgt'], label_col='income', 
                    event_label=">50K", encoding_configs={'bin_method': 'equal_frequency', 
                                                          'bin_size': 20, 'monotonicity_check': 0})
odf.toPandas()

attribute      iv
0        age  1.1592
1        sex  0.3037
2  workclass  0.1625
3       race  0.0697
4     fnlwgt  0.0160

In [25]:
# Example 6 - selected columns + encoding configs (monotonicity check )
odf = IV_calculation(spark, idf = df, list_of_cols= ['age','sex','race','workclass','fnlwgt'], label_col='income', 
                    event_label=">50K", encoding_configs={'bin_method': 'equal_frequency', 
                                                          'bin_size': 10, 'monotonicity_check': 1})
odf.toPandas()

attribute      iv
0        age  0.5842
1        sex  0.3037
2  workclass  0.1625
3       race  0.0697
4     fnlwgt  0.0087

# Information Gain (IG)
- API specification of function **IG_calculation** can be found <a href="https://docs.anovos.ai/api/data_analyzer/association_evaluator.html">here</a>
- Supports only binary target variable

In [26]:
from anovos.data_analyzer.association_evaluator import IG_calculation

In [27]:
# Example 1 - with mandatory arguments (rest arguments have default values)
odf = IG_calculation(spark, df, label_col='income', event_label=">50K")
odf.toPandas()

attribute      ig
0     relationship  0.1654
1   marital-status  0.1565
2              age  0.0935
3        education  0.0932
4       occupation  0.0931
5    education-num  0.0883
6   hours-per-week  0.0565
7     capital-gain  0.0428
8              sex  0.0372
9        workclass  0.0217
10    capital-loss  0.0119
11  native-country  0.0088
12            race  0.0086
13          fnlwgt  0.0011
14          logfnl  0.0006
15           empty  0.0000
16             ifa     NaN

In [28]:
# Example 2 - 'all' columns (excluding drop_cols)
odf = IG_calculation(spark, idf = df, list_of_cols='all', drop_cols=['ifa'], label_col='income', event_label=">50K")
odf.toPandas()

attribute      ig
0     relationship  0.1654
1   marital-status  0.1565
2              age  0.0935
3        education  0.0932
4       occupation  0.0931
5    education-num  0.0883
6   hours-per-week  0.0565
7     capital-gain  0.0428
8              sex  0.0372
9        workclass  0.0217
10    capital-loss  0.0119
11  native-country  0.0088
12            race  0.0086
13          fnlwgt  0.0011
14          logfnl  0.0006
15           empty  0.0000

In [29]:
# Example 3 - selected columns
odf = IG_calculation(spark, idf = df, list_of_cols= ['age','sex','race','workclass','fnlwgt'], label_col='income', event_label=">50K")
odf.toPandas()

attribute      ig
0        age  0.0935
1        sex  0.0372
2  workclass  0.0217
3       race  0.0086
4     fnlwgt  0.0011

In [30]:
# Example 4 - selected columns + encoding configs (bin method equal_range instead of default equal_frequency )
odf = IG_calculation(spark, idf = df, list_of_cols= ['age','sex','race','workclass','fnlwgt'], label_col='income', 
                    event_label=">50K", encoding_configs={'bin_method': 'equal_range', 
                                                          'bin_size': 10, 'monotonicity_check': 0})
odf.toPandas()

attribute      ig
0        age  0.0918
1        sex  0.0372
2  workclass  0.0217
3       race  0.0086
4     fnlwgt  0.0002

In [31]:
# Example 5 - selected columns + encoding configs (bin_size 20 instead of default 10 )
odf = IG_calculation(spark, idf = df, list_of_cols= ['age','sex','race','workclass','fnlwgt'], label_col='income', 
                    event_label=">50K", encoding_configs={'bin_method': 'equal_frequency', 
                                                          'bin_size': 20, 'monotonicity_check': 0})
odf.toPandas()

attribute      ig
0        age  0.0968
1        sex  0.0372
2  workclass  0.0217
3       race  0.0086
4     fnlwgt  0.0021

In [32]:
# Example 6 - selected columns + encoding configs (monotonicity check )
odf = IG_calculation(spark, idf = df, list_of_cols= ['age','sex','race','workclass','fnlwgt'], label_col='income', 
                    event_label=">50K", encoding_configs={'bin_method': 'equal_frequency', 
                                                          'bin_size': 10, 'monotonicity_check': 1})
odf.toPandas()

attribute      ig
0        age  0.0688
1        sex  0.0372
2  workclass  0.0217
3       race  0.0086
4     fnlwgt  0.0011